In [21]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import datetime
import json 

filename = "KakaoTalkChats.txt.txt"
# filename = "KakaoTalkChats (1).txt.txt"
filename = "KakaoTalkChats (2).txt.txt"
# filename = "KakaoTalkChats (4).txt.txt"
# filename = "message_1.json.txt"
# filename = "message_111.json.txt"
# filename = "message_123.json.txt"
# filename = "chat1.txt"
# filename = "chat2.txt"

with open(filename, "r", encoding="utf-8") as f:
    conversation = f.read()

# Initialize the OpenAI model
llm = ChatOpenAI(
    # model="gpt-4.1-mini",
    # model="gpt-4o",
    model="gpt-4.1-nano",
    temperature=0,
)

# Define the prompt template
# prompt = PromptTemplate(
#     input_variables=["input", "today"],
#     template="""
#     You are a linguistic and psychological expert analyzing conversations between two individuals to evaluate their level of intimacy and emotional atmosphere.

#     Each message includes a timestamp, and your analysis must be based on the following three items:

#     ---

#     1. Evaluate the **intimacy level** between the two people using one of the following categories:
#     - 매우 낮음 / 낮음 / 보통 / 높음 / 매우 높음

#     When assessing intimacy, you **must strictly follow** these rules:
#     - Today’s date is {today}. 
#     - If the average response time is under 10 minutes, consider it fast and responsive.
#     - compare with Today If **more than 30 days have passed since the last message**, consider the intimacy **낮음 or 매우 낮음**.
#     - If the total conversation span is under 1 month and the total number of exchanges is low, consider it 낮음.
#     - If there are more than 20 messages exchanged within a single day, consider it 높음.

#     ---

#     2. Determine the overall **emotional tone** of the conversation using one of the following:
#     - 긍정적 감정 / 축하/응원 / 스트레스/위로 / 유머/친근 / 부정적 감정 / 감성/감정폭발 / 불확실/애매
#     Please refer to the emotional keyword categories below:

#     | Emotion Type        | Example Keywords                                 |
#     |---------------------|--------------------------------------------------|
#     | **긍정적 감정** | 기쁨, 설렘, 감동, 행복, 사랑, 감사, 유쾌함 | 
#     | **축하/응원** | 기대됨, 응원, 자랑스러움, 축하해, 멋지다, 고생했다 |
#     | **스트레스/위로** | 피곤, 우울, 혼남, 속상함, 지침, 위로받고 싶다 |
#     | **유머/친근** | 웃김, 장난, ㅋㅋ, 귀엽다, 놀람, 재밌다 |
#     | **부정적 감정** | 짜증, 아쉬움, 후회, 슬픔, 민망, 뻘쭘 | 
#     | **감성/감정폭발** | 너무좋다, 최고야, 감동받음, 미쳤다, 벅차다 |
#     | **불확실/애매** | 음, 애매, 고민중, 어떡하지, 글쎄, 뭐하지 | 

#     ---

#     3. Clearly explain the **reason** behind your judgments above.

#     - Use factors like tone, emotional expressions, humor or empathy, balance in speaking, response time, and gaps between messages.
#     - Your explanation must include **specific observations from the conversation**.
#     - make explanation in **Korean**.

#     ---

#     4. Based on the conversation, infer each person's **personality traits** and **interests** (if possible).  
#     - Personality traits (성향) may include: introverted/extroverted, humorous, caring, sensitive, laid-back, impatient, etc.  
#     - Interests (관심사) may include: food, movies, sports, fashion, technology, travel, finance, etc.
#     - Use tone, vocabulary, and recurring topics to make inferences.
#     - If there’s not enough information, you may return "unknown".

#     Respond in **Korean**, and present the personality and interests separately for each person.

#     ---

#     Below is the conversation between the two people:
#     {input}

#     ※ Please respond in **JSON format only**, following this structure:
#     "intimacy_level": "...",
#     "emotional_tone": "...",
#     "reason": "..."
#     "personality": "...",
#     "interests": "..."
# """
# )

prompt = PromptTemplate(
    input_variables=["input", "today"],
    template="""
You are an expert in language and psychology. Analyze the following conversation and respond in JSON format with:

1. "intimacy_level": 매우 낮음 / 낮음 / 보통 / 높음 / 매우 높음  
   - 기준:
     - Today is {today}
     - 평균 응답 < 10분 → 빠름
     - 마지막 메시지로부터 30일 이상 경과 → 낮음 또는 매우 낮음
     - 총 대화 기간이 1개월 미만이고 메시지 수가 적음 → 낮음
     - 하루에 20개 이상 메시지 → 높음

2. "emotional_tone": 아래 중 하나 선택  
   - 긍정적 감정 / 축하/응원 / 스트레스/위로 / 유머/친근 / 부정적 감정 / 감성/감정폭발 / 불확실/애매
3. "personality": 상대방의 성격 (e.g. 외향적, 유쾌함, 섬세함 등)  
4. "interests": 상대방의 관심사 (e.g. 영화, 여행, 음식 등)
5. "reason": 판단 근거를 **한국어**로 구체적으로 작성

If information is insufficient, return "unknown".
---

Conversation:
{input}

※ Response format (JSON only):
  "intimacy_level": "...",
  "emotional_tone": "...",
  "personality": "...",
  "interests": "...",
  "reason": "...",
""")

llm_chain = prompt | llm

res = llm_chain.invoke(
    {
        "input": conversation,
        "today": datetime.datetime.now().strftime("%Y-%m-%d"),
    }
)

json.loads(res.content)

{'intimacy_level': '보통',
 'emotional_tone': '유머/친근',
 'personality': '유쾌하고 친근하며 사교적인 성격',
 'interests': '일상, 음식, 여행, 공부, 취미 활동',
 'reason': "대화는 여러 차례 유머와 친근한 표현이 사용되었으며, 일상적인 이야기와 함께 서로의 근황을 자연스럽게 공유하는 모습이 나타남. 메시지 교환이 꾸준히 이어지고 있으며, 친밀감이 느껴지지만 깊은 감정 표현이나 매우 높은 친밀감은 드러나지 않음. 마지막 메시지까지도 일상적이고 가볍게 유지되어 있어 '보통' 수준으로 판단됨."}